In [1]:
import pandas as pd

In [2]:
df_negocios=pd.read_csv('restaurantes por subcategoria.csv',encoding='utf-8',sep=',')

In [3]:
pd_reviews=pd.read_parquet('review_final_final.parquet', engine='pyarrow')

In [4]:
# Crear columna para sentimientos positivos
pd_reviews['positive_sentiment'] = pd_reviews['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)

# Crear columna para sentimientos negativos
pd_reviews['negative_sentiment'] = pd_reviews['sentiment'].apply(lambda x: 1 if x =='negative' else 0)
grouped_sentiments = pd_reviews.groupby('gmap_id').agg({
    'positive_sentiment': 'sum',
    'negative_sentiment': 'sum'
}).reset_index()

In [5]:
df_merged = pd.merge(df_negocios, grouped_sentiments, on='gmap_id', how='inner')

In [6]:
# Supongamos que df es tu DataFrame con los datos mencionados
df_merged['success'] = df_merged['avg_rating'].apply(lambda x: 1 if x >= 3.5 else 0)

In [ ]:
df_negocios

In [7]:
features = ['latitude', 'longitude', 'num_of_reviews', 'cluster', 'negative_sentiment','positive_sentiment']
X = df_merged[features]
y = df_merged['success']

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:

# Entrenar el modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(report)

Accuracy: 0.9330543933054394
              precision    recall  f1-score   support

           0       0.70      0.13      0.23       104
           1       0.94      1.00      0.97      1330

    accuracy                           0.93      1434
   macro avg       0.82      0.57      0.60      1434
weighted avg       0.92      0.93      0.91      1434



In [29]:
# Ejemplo de los datos de entrada para un nuevo negocio
new_data = {
    'latitude': 34.0580917,
    'longitude': -118.2921295,
    'num_of_reviews': 301,
    'cluster': 33,
    'negative_sentiment':300,
    'positive_sentiment':1
}

# Convertir los datos de entrada a un DataFrame
new_data_df = pd.DataFrame([new_data])

In [27]:
# Ejemplo de los datos de entrada para un nuevo negocio
new_data = {
    'latitude': 34.0580917,
    'longitude': -118.2921295,
    'num_of_reviews': 35,
    'cluster': 4,
    'negative_sentiment':5,
    'positive_sentiment':30
}

# Convertir los datos de entrada a un DataFrame
new_data_df = pd.DataFrame([new_data])

In [30]:
# Hacer la predicción
prediction = model.predict(new_data_df)

# Interpretar el resultado
if prediction[0] == 1:
    print("Predicción: El negocio será exitoso.")
else:
    print("Predicción: El negocio no será exitoso.")


Predicción: El negocio no será exitoso.
